In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os
import time

import snapshot_obj
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

# Massive compact non-luminous halos

Here, we inspect the properties of the clump of subhalos in the $\Lambda$CDM simulation with $v_\text{max} \sim v_\text{1kpc} > 35 \text{km/s}$.

In [ ]:
snap = snapshot_obj.Snapshot("V1_LR_fix", 127)
max_point = snap.get_subhalos("Max_Vcirc", "Extended")
vmax = max_point[:,0] * units.cm.to(units.km)
v1kpc = snap.get_subhalos("V1kpc", "Extended") * units.cm.to(units.km)

In [ ]:
masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
    snap, (1,0), (2,0))
mask_dark = dataset_compute.split_luminous(snap)[1]
mask_pecul = np.logical_and(vmax > 35, np.abs(vmax-v1kpc)/vmax < 0.1)
mask_pecul = np.logical_and.reduce([mask_isol, mask_dark, mask_pecul])
mask_ref = np.logical_and(vmax > 30, vmax < 50)
mask_ref = np.logical_and.reduce([mask_isol, mask_dark, mask_ref])

In [ ]:
print(np.sum(mask_pecul))
print(np.sum(mask_ref))
print(np.sum(np.logical_and(mask_pecul, mask_ref)))

In [ ]:
masstype = snap.get_subhalos("MassType")
print(masstype[mask])

In [ ]:
rot_curves = snap.get_subhalos('Vcirc', group='Extended/RotationCurve/All')
sub_offset = snap.get_subhalos('SubOffset', group='Extended/RotationCurve/All')

fig, axes = plt.subplots()

idxs = np.arange(mask_pecul.size)[mask_ref]
for idx in idxs:
    v_circ = rot_curves[sub_offset[idx]:sub_offset[idx+1],0] * units.cm.to(units.km)
    radii = rot_curves[sub_offset[idx]:sub_offset[idx+1],1] * units.cm.to(units.kpc)
    axes.plot(radii, v_circ, c='black')

idxs = np.arange(mask_pecul.size)[mask_pecul]
for idx in idxs:
    v_circ = rot_curves[sub_offset[idx]:sub_offset[idx+1],0] * units.cm.to(units.km)
    radii = rot_curves[sub_offset[idx]:sub_offset[idx+1],1] * units.cm.to(units.kpc)
    axes.plot(radii, v_circ, c='gray') 

In [ ]:
LG = dataset_compute.compute_LG_centre(snap, (1,0), (2,0))
dist_to_lg = dataset_compute.distance_to_point(snap, LG) * units.cm.to(units.kpc)
print(dist_to_lg[mask_pecul])